In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import networkx as nx
from torch_geometric.utils import from_networkx

import sys
sys.path.insert(0, "..")
from shared.GNNModel import GNNModel

def run_with_validation_and_early_stopping():
    # 加载数据
    loaded_data = torch.load('training_data.pt')
    X_tensor_loaded = loaded_data['X_tensor']  
    search_terms_tensor_loaded = loaded_data['search_terms_tensor']
    print("X_tensor_loaded: ", X_tensor_loaded.shape)
    labels_tensor_loaded = loaded_data['labels_tensor']

    max_label = labels_tensor_loaded.max().item()
    num_nodes = max_label + 1
    time_series_length = X_tensor_loaded.size(1)
    embedding_size = X_tensor_loaded.size(2)

    # 加载图
    gml_file = './pmfg_graph.graphml'
    G = nx.read_graphml(gml_file)
    data = from_networkx(G)
    if 'x' not in data:
        data.x = torch.rand(G.number_of_nodes(), embedding_size)
    edge_index = data.edge_index
    x = data.x

    # 将数据随机打乱并分为训练集和测试集 (80%训练+验证, 20%测试)
    total_samples = X_tensor_loaded.size(0)
    indices = torch.randperm(total_samples)
    test_ratio = 0.2
    test_size = int(total_samples * test_ratio)
    trainval_size = total_samples - test_size

    trainval_indices = indices[:trainval_size]
    test_indices = indices[test_size:]

    X_trainval = X_tensor_loaded[trainval_indices]
    search_trainval = search_terms_tensor_loaded[trainval_indices]
    y_trainval = labels_tensor_loaded[trainval_indices]

    X_test = X_tensor_loaded[test_indices]
    search_test = search_terms_tensor_loaded[test_indices]
    y_test = labels_tensor_loaded[test_indices]

    # 从训练+验证集中再分10%出来作为验证集
    val_ratio = 0.1
    val_size = int(trainval_size * val_ratio)
    train_size = trainval_size - val_size

    train_indices, val_indices = random_split(range(trainval_size), [train_size, val_size])
    train_indices = torch.tensor(train_indices.indices)
    val_indices = torch.tensor(val_indices.indices)

    X_train = X_trainval[train_indices]
    search_train = search_trainval[train_indices]
    y_train = y_trainval[train_indices]

    X_val = X_trainval[val_indices]
    search_val = search_trainval[val_indices]
    y_val = y_trainval[val_indices]

    # 构建 DataLoader
    train_dataset = TensorDataset(search_train, X_train, y_train)
    val_dataset = TensorDataset(search_val, X_val, y_val)
    test_dataset = TensorDataset(search_test, X_test, y_test)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型、优化器、损失函数
    model = GNNModel(num_nodes=num_nodes, embedding_size=embedding_size, dropout=0.5)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    num_epochs = 50
    best_val_loss = float('inf')
    patience = 5
    no_improve_count = 0

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # 训练
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for search_batch, time_series_batch, label_batch in train_loader:
            optimizer.zero_grad()
            print(search_batch.shape, time_series_batch.shape, edge_index.shape, x.shape)
            output = model(search_batch, time_series_batch, edge_index, x)
            loss = criterion(output, label_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(output, 1)
            correct += (predicted == label_batch).sum().item()
            total += label_batch.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # 验证
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for search_batch, time_series_batch, label_batch in val_loader:
                output = model(search_batch, time_series_batch, edge_index, x)
                loss = criterion(output, label_batch)
                val_loss += loss.item()
                _, predicted = torch.max(output, 1)
                val_correct += (predicted == label_batch).sum().item()
                val_total += label_batch.size(0)

        val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Early Stopping 检查
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve_count = 0
            # 保存最佳模型参数
            torch.save(model.state_dict(), "best_model.pth")
        else:
            no_improve_count += 1
            if no_improve_count >= patience:
                print("Early stopping triggered!")
                break

    print("Training complete.")

    # 绘制训练和验证的Loss与Accuracy
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(train_accuracies, label='Train Acc')
    plt.plot(val_accuracies, label='Val Acc')
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # 返回必要的变量以便后续使用
    return model, test_loader, edge_index, x, num_nodes, time_series_length, embedding_size

# 运行训练并获取返回的变量
model, test_loader, edge_index, x, num_nodes, time_series_length, embedding_size = run_with_validation_and_early_stopping()


X_tensor_loaded:  torch.Size([2907, 10, 1536])
torch.Size([32, 1536]) torch.Size([32, 10, 1536]) torch.Size([2, 1626]) torch.Size([287, 1536])
torch.Size([32, 1536]) torch.Size([32, 10, 1536]) torch.Size([2, 1626]) torch.Size([287, 1536])
torch.Size([32, 1536]) torch.Size([32, 10, 1536]) torch.Size([2, 1626]) torch.Size([287, 1536])


KeyboardInterrupt: 

### This code is for testing, with "hit rate", which means the top 10 out of 300+ courses recommendation contains the real visit one. And this rate based on the saved model is 86.6% 

In [6]:
import torch
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

def evaluate_hit_rate_k(model, test_loader, edge_index, x, k=10):
    model.eval()
    hits = 0
    total = 0
    with torch.no_grad():
        for search_batch, time_series_batch, label_batch in test_loader:
            output = model(search_batch, time_series_batch, edge_index, x)
            _, topk_indices = torch.topk(output, k, dim=1)

            for i in range(label_batch.size(0)):
                label = label_batch[i].item()
                recommended_list = topk_indices[i].tolist()
                if label in recommended_list:
                    hits += 1
                total += 1

    hit_rate = hits / total if total > 0 else 0.0
    return hit_rate

def run_evaluation():

    # 初始化模型并加载参数
    model = GNNModel(num_nodes=num_nodes, time_series_length=10, embedding_size=embedding_size)
    model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))
    model.eval()

    # 计算 Hit Rate@10
    hit_rate_10 = evaluate_hit_rate_k(model, test_loader, edge_index, x, k=10)
    print(f"Hit Rate@10: {hit_rate_10:.4f}")

if __name__ == "__main__":
    run_evaluation()


NameError: name 'num_nodes' is not defined